<a href="https://colab.research.google.com/github/sofia4009/Oral-disease/blob/main/Train_Test_GoogleDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Sofia
Date: March 26
Subject: Training and getting experience with deep learning
Classification of dataset from kaggle:  https://www.kaggle.com/datasets/salmansajid05/oral-diseases
  - use stratified k-fold cross validation (with k = 5) to split dataset A
  - use the following deep neural networks for training, testing and comparing their performance
  - (do not use any data augmentation for now; just normalise all image pixel values to the range [0, 1];
  - resize all images to dimensions 112x112x3 for speeding up the training;
  - use categorical cross entropy as the loss function and f1 score as evaluation metric;
  - use the pretrained models on ImageNet):
    - ResNet18, ResNet50, ConvNeXt, EfficientNetB0, Transformers (i.e., ViT)

#Data Preparation

In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as xlImage
from openpyxl import Workbook
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import h5py
import time

In [3]:
# Initializations
image_size = (112,112)

# Specify the root directory where your images are located
main_DS_directory = '/content/New_Dataset/DataSet B' #'/content/oral-diseases'

# Specify the output directory to save the processed images
Preprocessed_DS_directory = '/content/DataSet B' #'/content/decreased_oral_diseases'

# Specify the output directory to save the excel file
results_directory = '/content/drive/My Drive/QM/results.xlsx'
Plot_directory = '/content/drive/My Drive/QM/plots.xlsx'


In [4]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

with zipfile.ZipFile('/content/drive/MyDrive/QM/New_Dataset.zip') as zipObj:
  members = [file for file in zipObj.namelist() if "__MACOSX" not in file]
  zipObj.extractall('/content/New_Dataset', members=members)

# Optionally, remove the __MACOSX directory if it was extracted
macosx_folder = os.path.join('/content/New_Dataset', "__MACOSX")
if os.path.exists(macosx_folder):
    os.rmdir(macosx_folder)


Mounted at /content/drive/


In [5]:
# Normalizing them to [0,1], Resizing to 112,112, 3
def normalize_and_resize_image(image, target_size):

    # Resize image into 112*112*3
    image = image.resize(target_size)

    # Normalize pixel values to the range [0, 1]
    image = np.array(image)
    image = image / 255.0

    # Convert the normalized numpy array back to PIL image
    image = Image.fromarray((image * 255).astype(np.uint8))

    return image

# Saving the normalized images into a new directory in Google colab with the same subdirectories and structure
def process_images_in_directory(directory, Preprocessed_DS_directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has an image extension
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                # Construct the full path to the input image file
                image_path = os.path.join(root, file)
                if 'Decay' not in image_path and 'Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset' not in image_path:
                  # Open the image using PIL
                  image = Image.open(image_path)

                  # Ensure image is in RGB mode
                  image = image.convert("RGB")

                  # Normalize and Resize the image
                  #print(image_path)
                  processed_image = normalize_and_resize_image(image, image_size)

                  # Construct the full path to the output directory
                  output_subdirectory = os.path.relpath(root, directory)
                  output_path = os.path.join(Preprocessed_DS_directory, output_subdirectory)
                  os.makedirs(output_path, exist_ok=True)

                  # Save the processed image
                  filename = os.path.join(output_path, file)
                  processed_image.save(filename, format='JPEG')  # Change 'JPEG' to the desired format
                  #print(filename)
                  #if filename.find('augmented') == -1 or filename.find('Caries_Gingivitus_ToothDiscoloration_Ulcer-yolo_annotated-Dataset') == -1:
                  #np.save(filename, image)

In [6]:
# Call the function to process images in the directory
process_images_in_directory(main_DS_directory, Preprocessed_DS_directory)

In [7]:
file_count = sum(len(files) for _, _, files in os.walk(main_DS_directory))
print(f"Number of Original Images: {file_count}")

file_count = sum(len(files) for _, _, files in os.walk(Preprocessed_DS_directory))
print(f"Number of Preprocessed Images: {file_count}")

Number of Original Images: 539
Number of Preprocessed Images: 532


# Training the dataset Using Pre-trained models on ImageNet

# Initializations for training

In [8]:
# Define transformations by Composing several transforms together
transform = transforms.Compose([
    transforms.Resize(image_size),
    #Convert a PIL Image or ndarray to tensor
    transforms.ToTensor()
])

# Use ImageFolder to load your dataset
dataset = torchvision.datasets.ImageFolder(root = Preprocessed_DS_directory, transform=transform)

# Define number of classes and list of labels in the dataset
num_classes = len(dataset.classes)
classes = dataset.classes

# Define k-fold for cross-validation
k_folds = 5

# Seed (random_state) is set to initialize the random number generator while splitting the DataSet into k folds
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Define hyperparameters to search over
learning_rates = [0.001, 0.0001]
batch_sizes = [32, 64, 128]
optimizers = ['Adam', 'SGD', 'RMSprop']

epochs = 35

best_model = None
best_f1 = 0.0
best_hyperparameters = None

# Define the path where you want to save the model weights in Google Drive
checkpoint_path = '/content/drive/My Drive/QM/modelDSBfold1.h5'


In [9]:
#print(classes)
class_counts = {}
for class_name in os.listdir(Preprocessed_DS_directory):
    # Construct the full path to the class directory
    class_directory = os.path.join(Preprocessed_DS_directory, class_name)

    # Check if the path is a directory
    if os.path.isdir(class_directory):
        # Count the number of files in the class directory
        num_files = len(os.listdir(class_directory))

        # Store the count in the class_counts dictionary
        class_counts[class_name] = num_files

# ResNet18

In [ ]:
"""# ResNet18"""

from torchvision.models import resnet18

# An empty list for storing the results
results = []
f1_vals = [0] * k_folds
results.append({'Model': 'ResNet18'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

epochs = 10
num_epochs = epochs

for lr in learning_rates:
    for batch_size in batch_sizes:
      for optimizer_name in optimizers:
        print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.imgs, dataset.targets)):
            train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

            train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

            best_f1 = 0
            # Define the device (GPU if available, otherwise CPU)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(torch.cuda.is_available())

            # Define the neural network
            model = resnet18(weights='ResNet18_Weights.DEFAULT')
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)
            model = model.to(device) # Move the model to GPU

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                # Training loop
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                  for inputs, labels in val_loader:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total += labels.size(0)
                      correct += (preds == labels).sum().item()
                      all_preds.extend(preds.cpu().numpy())
                      all_labels.extend(labels.cpu().numpy())
                epoch_val_loss = running_loss / len(val_loader)
                epoch_val_accuracy = correct / total
                val_losses.append(epoch_val_loss)
                val_accuracies.append(epoch_val_accuracy)

                # Calculate F1 score
                f1 = f1_score(all_labels, all_preds, average='macro')
                print(f"Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}")

                # Update best F1 score and model
                if f1 > best_f1:
                  best_f1 = f1

            f1_vals[fold] = best_f1
            results.append({
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1
              })
        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })



In [ ]:
# ----------------- Save the results in a excel file ---------------------
# Save the results as a DataFrame to be saved in an Excel file
df = pd.DataFrame(results)

# Load an existing Excel file
if os.path.isfile(results_directory):
  existing_file = pd.read_excel(results_directory)
  # Append the new DataFrame to the existing file
  df = pd.concat([existing_file, df])

# Write the DataFrame to the excel file in the directory
df.to_excel(results_directory, index=False)
# -------------- End of Save the results in a excel file ----------------

# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active

#ResNet50

In [ ]:

"""#ResNet50"""

from torchvision.models import resnet50

# An empty list for storing the results
results = []
f1_vals = [0] * k_folds
results.append({'Model': 'ResNet50'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

epochs = 10
# Define hyperparameters to search over
learning_rates = [0.0001]
batch_sizes = [32]
optimizers = ['Adam', 'SGD', 'RMSprop']
num_epochs = epochs

for lr in learning_rates:
    for batch_size in batch_sizes:
      for optimizer_name in optimizers:
        print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.imgs, dataset.targets)):
            train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

            train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

            best_f1 = 0
            # Define the device (GPU if available, otherwise CPU)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            # Define the neural network
            model = resnet50(weights='ResNet50_Weights.DEFAULT')
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)
            model = model.to(device) # Move the model to GPU

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                # Training loop
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                  for inputs, labels in val_loader:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total += labels.size(0)
                      correct += (preds == labels).sum().item()
                      all_preds.extend(preds.cpu().numpy())
                      all_labels.extend(labels.cpu().numpy())
                epoch_val_loss = running_loss / len(val_loader)
                epoch_val_accuracy = correct / total
                val_losses.append(epoch_val_loss)
                val_accuracies.append(epoch_val_accuracy)

                # Calculate F1 score
                f1 = f1_score(all_labels, all_preds, average='macro')
                print(f"Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}")

                # Update best F1 score and model
                if f1 > best_f1:
                  best_f1 = f1

            f1_vals[fold] = best_f1
            results.append({
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1
              })
        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })

In [ ]:
# ----------------- Save the results in a excel file ---------------------
# Save the results as a DataFrame to be saved in an Excel file
df = pd.DataFrame(results)

# Load an existing Excel file
if os.path.isfile(results_directory):
  existing_file = pd.read_excel(results_directory)
  # Append the new DataFrame to the existing file
  df = pd.concat([existing_file, df])

# Write the DataFrame to the excel file in the directory
df.to_excel(results_directory, index=False)
# -------------- End of Save the results in a excel file ----------------

# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active

Training with optimizer: Adam, learning rate: 0.0001, batch size: 32


# ConvNeXt

In [ ]:
"""# ConvNeXt"""

# An empty list for storing the results
results = []
f1_vals = [0] * k_folds
results.append({'Model': 'ConvNeXt'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = epochs

for lr in learning_rates:
    for batch_size in batch_sizes:
      for optimizer_name in optimizers:
        print("Training with optimizer: " + optimizer_name + ", learning rate: " + str(lr) + ", batch size: " + str(batch_size))

#        print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.imgs, dataset.targets)):
            train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

            #train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            #val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

            train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, pin_memory=True)
            val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler, pin_memory=True)

            best_f1 = 0

            # Define the neural network

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = torchvision.models.convnext_small(input_shape=image_size, num_classes=num_classes).to(device)

            #model = torchvision.models.convnext_small(input_shape=image_size, num_classes=num_classes)

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                # Training loop
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                for inputs, labels in train_loader:

                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                   for inputs, labels in val_loader:

                      inputs, labels = inputs.to(device), labels.to(device)
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total += labels.size(0)
                      correct += (preds == labels).sum().item()
                      all_preds.extend(preds.cpu().numpy())
                      all_labels.extend(labels.cpu().numpy())
                epoch_val_loss = running_loss / len(val_loader)
                epoch_val_accuracy = correct / total
                val_losses.append(epoch_val_loss)
                val_accuracies.append(epoch_val_accuracy)

                # Calculate F1 score
                f1 = f1_score(all_labels, all_preds, average='macro')
                #print(f"Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}")
                print("Fold " + str(fold+1) + ", Epoch " + str(epoch+1) + ", F1 Score: " + str(f1))
                # Update best F1 score and model
                if f1 > best_f1:
                  best_f1 = f1

            f1_vals[fold] = best_f1
            results.append({
                    'model': 'ConvNeXt_small',
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1
                    })
        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })

        # Save the results as a DataFrame to be saved in an Excel file
        df = pd.DataFrame(results)

        # Load an existing Excel file
        if os.path.isfile(results_directory):
            existing_file = pd.read_excel(results_directory)
            # Append the new DataFrame to the existing file
            df = pd.concat([existing_file, df])

        # Write the DataFrame to the excel file in the directory
        df.to_excel(results_directory, index=False)
# -------------- End of Save the results in a excel file ----------------



In [ ]:
# ----------------- Save the results in a excel file ---------------------
# Save the results as a DataFrame to be saved in an Excel file
df = pd.DataFrame(results)

# Load an existing Excel file
if os.path.isfile(results_directory):
  existing_file = pd.read_excel(results_directory)
  # Append the new DataFrame to the existing file
  df = pd.concat([existing_file, df])

# Write the DataFrame to the excel file in the directory
df.to_excel(results_directory, index=False)
# -------------- End of Save the results in a excel file ----------------

# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active

# EfficientNetB0

In [ ]:
"""# EfficientNetB0"""

! pip install --upgrade efficientnet-pytorch

from efficientnet_pytorch import EfficientNet

# An empty list for storing the results
results = []
f1_vals = [0] * k_folds
results.append({'Model': 'EfficientNetB0'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = epochs

for lr in learning_rates:
    for batch_size in batch_sizes:
      for optimizer_name in optimizers:
        print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.imgs, dataset.targets)):
            train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

            train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

            best_f1 = 0
            # Define the device (GPU if available, otherwise CPU)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            # Define the neural network
            model = EfficientNet.from_pretrained('efficientnet-b0', num_classes).to(device)  # Move the model to GPU

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                # Training loop
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                  for inputs, labels in val_loader:
                      inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                      outputs = model(inputs)
                      loss = criterion(outputs, labels)
                      running_loss += loss.item()
                      _, preds = torch.max(outputs, 1)
                      total += labels.size(0)
                      correct += (preds == labels).sum().item()
                      all_preds.extend(preds.cpu().numpy())
                      all_labels.extend(labels.cpu().numpy())
                epoch_val_loss = running_loss / len(val_loader)
                epoch_val_accuracy = correct / total
                val_losses.append(epoch_val_loss)
                val_accuracies.append(epoch_val_accuracy)

                # Calculate F1 score
                f1 = f1_score(all_labels, all_preds, average='macro')
                print(f"Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}")

                # Update best F1 score and model
                if f1 > best_f1:
                  best_f1 = f1

            f1_vals[fold] = best_f1
            results.append({
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1
              })
        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })



In [ ]:

# ----------------- Save the results in a excel file ---------------------
# Save the results as a DataFrame to be saved in an Excel file
df = pd.DataFrame(results)

# Load an existing Excel file
if os.path.isfile(results_directory):
  existing_file = pd.read_excel(results_directory)
  # Append the new DataFrame to the existing file
  df = pd.concat([existing_file, df])

# Write the DataFrame to the excel file in the directory
df.to_excel(results_directory, index=False)
# -------------- End of Save the results in a excel file ----------------

# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active

#Transformer: ViT

In [ ]:
# Define transformations by Composing several transforms together
# and change th image_size to (224*224) to be compatible with ViT
transform = transforms.Compose([
    transforms.Resize(224*224),
    #Convert a PIL Image or ndarray to tensor
    transforms.ToTensor()
])

# Use ImageFolder to load your dataset
dataset = torchvision.datasets.ImageFolder(root = Preprocessed_DS_directory, transform=transform)

# Define number of classes and list of labels in the dataset
num_classes = len(dataset.classes)
classes = dataset.classes

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
"""#Transformer: ViT"""

from transformers import ViTModel, ViTConfig

# Define transformations by Composing several transforms together
# and change th image_size to (224*224) to be compatible with ViT
transform = transforms.Compose([
    transforms.Resize(224*224),
    #Convert a PIL Image or ndarray to tensor
    transforms.ToTensor()
])

# Use ImageFolder to load your dataset
dataset = torchvision.datasets.ImageFolder(root = Preprocessed_DS_directory, transform=transform)

# Define number of classes and list of labels in the dataset
num_classes = len(dataset.classes)
classes = dataset.classes

# An empty list for storing the results
results = []
f1_vals = [0] * k_folds
results.append({'Model': 'Transformr: ViT'})

# Initialize lists to store training and validation losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = epochs

for batch_size in batch_sizes:
    for lr in learning_rates:
      for optimizer_name in optimizers:
        #print(f"Training with optimizer: {optimizer_name}, learning rate: {lr}, batch size: {batch_size}")
        print("Training with optimizer: " + optimizer_name + ", learning rate: " + str(lr) + ", batch size: " + str(batch_size))
        for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.imgs, dataset.targets)):
            train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
            val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

            train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

            best_f1 = 0

            # Define the device (GPU if available, otherwise CPU)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            # Define the neural network
            config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')
            model = ViTModel(config).to(device)  # Move the model to GPU

            #model = ViTModel(config)

            # Define loss function
            criterion = nn.CrossEntropyLoss()

            # Define optimizer
            if optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=lr)

            # Train and evaluate
            for epoch in range(num_epochs):
                # Training loop
                model.train()
                running_loss = 0.0
                correct = 0
                total = 0
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to GPU
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                epoch_train_loss = running_loss / len(train_loader)
                epoch_train_accuracy = correct / total
                train_losses.append(epoch_train_loss)
                train_accuracies.append(epoch_train_accuracy)

                # Validation loop
                model.eval()
                all_preds = []
                all_labels = []
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                  for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to GPU
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (preds == labels).sum().item()
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                epoch_val_loss = running_loss / len(val_loader)
                epoch_val_accuracy = correct / total
                val_losses.append(epoch_val_loss)
                val_accuracies.append(epoch_val_accuracy)

                # Calculate F1 score
                f1 = f1_score(all_labels, all_preds, average='macro')
                #print(f"Fold {fold+1}, Epoch {epoch+1}, F1 Score: {f1}")
                print("Fold " + str(fold+1) + ", Epoch " + str(epoch+1) + ", F1 Score: " + str(f1))
                # Update best F1 score and model
                if f1 > best_f1:
                  best_f1 = f1

            f1_vals[fold] = best_f1
            results.append({
                    'Optimizer': optimizer_name,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'Fold': fold+1,
                    'F1 Score': best_f1
              })
        results.append({
                  'min f1': min(f1_vals),
                  'max f1': max(f1_vals),
                  'average f1': sum(f1_vals)/len(f1_vals)
                  })

        # Define the filename for the Excel file
        excel_filename = Plot_directory

        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

        # Check if the Excel file exists
        if os.path.exists(excel_filename):
            # Load existing Excel file
            wb = load_workbook(excel_filename)
            ws = wb.active
        else:
            # Create a new Excel workbook
            wb = Workbook()
            ws = wb.active


In [ ]:
        # Save the results as a DataFrame to be saved in an Excel file
        df = pd.DataFrame(results)

        # Load an existing Excel file
        if os.path.isfile(results_directory):
            existing_file = pd.read_excel(results_directory)
            # Append the new DataFrame to the existing file
            df = pd.concat([existing_file, df])

        # Write the DataFrame to the excel file in the directory
        df.to_excel(results_directory, index=False)
# -------------------- Plot the error/epoch plot ------------------------

# Define the filename for the Excel file
excel_filename = Plot_directory

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(excel_filename), exist_ok=True)

# Check if the Excel file exists
if os.path.exists(excel_filename):
    # Load existing Excel file
    wb = load_workbook(excel_filename)
    ws = wb.active
else:
    # Create a new Excel workbook
    wb = Workbook()
    ws = wb.active